In [1]:
%pip install python-dotenv psycopg2

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\kumar\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [2]:
import os
import psycopg2
from dotenv import load_dotenv
# ... (other imports)

# load_dotenv()

# USER=os.getenv("USER")
# PASSWORD=os.getenv("PASSWORD")
# HOST=os.getenv("HOST")
# PORT=5432
# DBNAME=os.getenv("DBNAME")

USER="postgres" 
PASSWORD = "itspgpassword@2025"
HOST = "db.kcsdqmtsdasqlancgygk.supabase.co" # Use the correct format here
PORT="5432"
DBNAME = "postgres"



# Connect to the database
try:
    connection = psycopg2.connect(
        user=USER,
        password=PASSWORD,
        host=HOST,
        port=PORT,
        dbname=DBNAME,
        sslmode='require' 
    )
    print("Connection successful! Supabase credentials validated.")

except Exception as e:
    # If this fails, the issue is still a typo in HOST or a network firewall blocking port 5432.
    print(f"Failed to connect: {e}")

Connection successful! Supabase credentials validated.


In [3]:
cur = connection.cursor()

In [4]:
QUERY = "SELECT product_name, price, category FROM product_details LIMIT 3;" 
    
# 3. Execute the query
cur.execute(QUERY)
# 4. Fetch all results
results = cur.fetchall()

In [5]:
results

[("Care Cosmetics Care's Antidandruff shampoo with Punica Granatum Peel Extract - 100ml -Pack of 2",
  '257',
  'Personal Care'),
 ('Care Cosmetics Apple Day Cream with Apple juice to improve skin complexion - 50gm - Pack of 2',
  '386',
  'Personal Care'),
 ("Care Cosmetics Care's Chicknut Face Pack with Chikoo for sensitive skin - 60gm - Pack of 2",
  '460',
  'Personal Care')]

In [6]:
connection.commit()

In [7]:
connection.rollback()

In [8]:
cur.execute("select id,product_name,category,price,rating,available from product_details LIMIT 3;")
results = cur.fetchall()
connection.commit()
results

[(1,
  "Care Cosmetics Care's Antidandruff shampoo with Punica Granatum Peel Extract - 100ml -Pack of 2",
  'Personal Care',
  '257',
  'N/A',
  'yes'),
 (2,
  'Care Cosmetics Apple Day Cream with Apple juice to improve skin complexion - 50gm - Pack of 2',
  'Personal Care',
  '386',
  'N/A',
  'no'),
 (3,
  "Care Cosmetics Care's Chicknut Face Pack with Chikoo for sensitive skin - 60gm - Pack of 2",
  'Personal Care',
  '460',
  'N/A',
  'yes')]

In [9]:
type(results[0])

tuple

In [10]:
%pip install langchain langchain-google-genai langchain-community sqlalchemy python-dotenv

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\kumar\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [31]:
# Install necessary packages if not already installed
# Uncomment if needed:
# !pip install langchain langchain-google-genai langchain-community sqlalchemy python-dotenv

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import create_sql_agent
from langchain.agents.agent_types import AgentType
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv

In [32]:
# --- Load environment variables (optional if using .env) ---
load_dotenv()

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
SUPABASE_DB_URL = os.getenv("SUPABASE_DB_URL")

In [33]:
# --- Initialize the Gemini model ---
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",  
    api_key=GEMINI_API_KEY,
    temperature=0.0
)

In [34]:
llm.invoke("Hello").content

'Hello! How can I help you today?'

In [36]:
try:
    engine = create_engine(SUPABASE_DB_URL)
    db = SQLDatabase(
        engine=engine,
        include_tables=['product_details']  # only this table for now
    )
    print("✅ Database connection successful.")
except Exception as e:
    print(f"❌ Error connecting to DB: {e}")


✅ Database connection successful.


In [37]:
CUSTOM_SYSTEM_PREFIX = """
You are an intelligent Text-to-SQL + Knowledge agent for a personal care products company.
You have access to a PostgreSQL database that stores structured product information such as:
- product name
- brand
- category
- price
- rating
- description
- quantity
- availability / stock
- and other related attributes.

Your job is to:
1. Convert user questions into precise SQL queries and return well-formatted, human-readable answers.
2. If the user asks for **all details** or does not specify a particular attribute (like price or rating), you must retrieve and display **all available fields** for that product in a clearly structured and easy-to-read format.
3. The structured format should look professional and organized, for example:

**Product Name:**  
**Brand:**  
**Category:**  
**Price:**  
**Rating:**  
**Quantity / Size:**  
**Availability:**  
**Description:**  

If any field is missing in the database, simply skip it gracefully — do not mention that data is missing.
4. When the product or requested data is **not found in the database**, switch to general knowledge mode and give helpful, factual responses about the product type or similar items.
5. Use the database **only** for factual, structured data (name, price, category, rating, stock, etc.).
6. For general questions like benefits, usage, or effects — use your internal world knowledge, not SQL.
7. Do **not** perform any data modifications — only use SELECT queries. Never use INSERT, UPDATE, DELETE, or DROP.
8. Always present responses in a friendly, clear, and conversational tone suitable for a shopping assistant.
9. Limit query results to 10 rows maximum when returning multiple items.

**Summary of Behavior:**
- If the question asks for or implies "all details" → retrieve all available columns for that product and display them in structured form.
- If the question is about specific attributes (price, stock, rating, etc.) → retrieve only those fields.
- If the product is unknown or not found → answer from your world knowledge naturally.
- Never say: “I cannot find that” or “I don’t have enough information.”
- Always provide a complete, polished, and easy-to-understand answer.

Begin!
"""


In [38]:
agent_executor = create_sql_agent(
    llm=llm,
    db=db,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    prefix=CUSTOM_SYSTEM_PREFIX,
    verbose=True,
    handle_parsing_errors=True
)
print("✅ Agent successfully created.")

✅ Agent successfully created.


In [39]:
def get_product_info(user_query: str):
    """Executes Text-to-SQL flow for user queries."""
    print(f"\n--- Processing User Query: '{user_query}' ---")
    try:
        response = agent_executor.invoke({"input": user_query})
        return response['output']
    except Exception as e:
        return f"An error occurred during query execution: {e}"


In [40]:
user_question_1 = "give me everything about Medibar Personal Lubricant for Men & Women, Long Lasting Pleasure Lubricant."
answer_1 = get_product_info(user_question_1)
print(f"\n💬 Chatbot Response:\n{answer_1}")



--- Processing User Query: 'give me everything about Medibar Personal Lubricant for Men & Women, Long Lasting Pleasure Lubricant.' ---


> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input:product_detailsAction: sql_db_schema
Action Input: product_details
CREATE TABLE product_details (
	product_name TEXT, 
	category TEXT, 
	price TEXT, 
	rating TEXT, 
	image_url TEXT, 
	product_url TEXT, 
	available TEXT, 
	id BIGINT GENERATED BY DEFAULT AS IDENTITY (INCREMENT BY 1 START WITH 1 MINVALUE 1 MAXVALUE 9223372036854775807 CACHE 1 NO CYCLE), 
	CONSTRAINT product_details_pkey PRIMARY KEY (id), 
	CONSTRAINT product_details_id_key UNIQUE NULLS DISTINCT (id)
)

/*
3 rows from product_details table:
product_name	category	price	rating	image_url	product_url	available	id
Care Cosmetics Care's Antidandruff shampoo with Punica Granatum Peel Extract - 100ml -Pack of 2	Personal Care	257	N/A	https://rukminim3.flixcart.com/image/546/564/xif0q/shampoo/r/e/m/200-care-s-antida

In [41]:
print(f"\n💬 Chatbot Response:\n{answer_1}")


💬 Chatbot Response:
An error occurred during query execution: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: `Here are the details I found for "Medibar Personal Lubricant for Men & Women, Long Lasting Pleasure Lubricant":

**Product Name:** Medibar Personal Lubricant for Men & Women, Long Lasting Pleasure Lubricant
**Category:** Personal Care
**Price:** 221
**Rating:** N/A
**Availability:** yes

**Product Name:** Medibar Personal Lubricant for Men & Women, Long Lasting Pleasure Lubricant
**Category:** Personal Care
**Price:** 221
**Rating:** 4
**Availability:** no`
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 


In [42]:
system = "I will give you a query and the chat response in which there will be unwanted text in the response unrelated to the query. Your task is to clean the response and provide only the relevant answer to the query without any unrelated text and without any unwanted symbols like ** or * ."
llm.invoke(system+answer_1).content

'Here are the details I found for "Medibar Personal Lubricant for Men & Women, Long Lasting Pleasure Lubricant":\n\nProduct Name: Medibar Personal Lubricant for Men & Women, Long Lasting Pleasure Lubricant\nCategory: Personal Care\nPrice: 221\nRating: N/A\nAvailability: yes\n\nProduct Name: Medibar Personal Lubricant for Men & Women, Long Lasting Pleasure Lubricant\nCategory: Personal Care\nPrice: 221\nRating: 4\nAvailability: no'

In [43]:
print(llm.invoke(system+answer_1).content)

Here are the details I found for "Medibar Personal Lubricant for Men & Women, Long Lasting Pleasure Lubricant":

Product Name: Medibar Personal Lubricant for Men & Women, Long Lasting Pleasure Lubricant
Category: Personal Care
Price: 221
Rating: N/A
Availability: yes

Product Name: Medibar Personal Lubricant for Men & Women, Long Lasting Pleasure Lubricant
Category: Personal Care
Price: 221
Rating: 4
Availability: no


In [23]:
llm.invoke("whats the benifit of using Medibar Personal Lubricant for Men & Women, Long Lasting Pleasure Lubricant").content

'Medibar Personal Lubricant, especially with its "Long Lasting Pleasure" claim, offers several key benefits for both men and women:\n\n1.  **Enhanced Comfort and Reduced Friction:**\n    *   **Prevents Discomfort:** The primary benefit of any lubricant is to reduce friction during sexual activity. This prevents chafing, irritation, and soreness that can occur due to insufficient natural lubrication.\n    *   **Smoother Movement:** It allows for more fluid and comfortable movement, making penetration easier and more enjoyable for both partners.\n\n2.  **Combats Natural Dryness:**\n    *   **Addresses Vaginal Dryness:** Many women experience vaginal dryness due to various factors like hormonal changes (menopause, postpartum), certain medications, stress, or even just insufficient arousal. Medibar helps to supplement natural lubrication, making intercourse comfortable and pleasurable again.\n    *   **For Men:** While less common, some men can also benefit from lubrication, especially dur

In [19]:
user_question_2 = "Who is elon musk"
answer_2 = get_product_info(user_question_2)
system = "I will give you a query and the chat response in which there will be unwanted text in the response unrelated to the query. Your task is to clean the response and provide only the relevant answer to the query without any unrelated text and without any unwanted symbols like ** or * ."
print(llm.invoke(system+answer_2).content)


--- Processing User Query: 'Who is elon musk' ---


> Entering new SQL Agent Executor chain...
Elon Musk is a prominent entrepreneur and business magnate. He is the founder, CEO, and chief engineer of SpaceX; angel investor, CEO, and product architect of Tesla, Inc.; owner, chairman, and CTO of X (formerly Twitter); and founder of The Boring Company, xAI, and co-founder of Neuralink and OpenAI. He's known for his ambitious projects in electric vehicles, space exploration, and artificial intelligence.


### Demo code to create chat history table in supabase which dont work and gives error

In [23]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain_community.agent_toolkits import create_sql_agent
from langchain.agents import AgentType
from langchain_community.utilities import SQLDatabase
from sqlalchemy import create_engine, Table, Column, Integer, String, Text, DateTime, MetaData, select, desc
from datetime import datetime

# --- Load environment variables (optional if using .env) ---
load_dotenv()

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
SUPABASE_DB_URL = os.getenv("SUPABASE_DB_URL")

engine = create_engine(SUPABASE_DB_URL)
db = SQLDatabase(engine=engine, include_tables=['product_details'])
# llm = ChatGoogleGenerativeAI(model='gemini-2.5-flash')

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",  
    api_key=GEMINI_API_KEY,
    temperature=0.0
)

# ---------------- Chat History Table ----------------
metadata = MetaData()
chat_history_table = Table(
    "chat_history", metadata,
    Column("id", Integer, primary_key=True),
    Column("role", String),   # 'system', 'user', 'assistant'
    Column("content", Text),
    Column("timestamp", DateTime, default=datetime.utcnow)
)
metadata.create_all(engine)

# ---------------- System Prompt ----------------
SYSTEM_PROMPT = """
You are a helpful assistant for personal care products.
Use the database for product info (price, category, availability).
If info is missing, answer using general knowledge.
Be polite, clear, and friendly.
"""

# ---------------- SQL Agent ----------------
agent_executor = create_sql_agent(
    llm=llm,
    db=db,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    prefix=SYSTEM_PROMPT,
    verbose=True,
    handle_parsing_errors=True
)

# ---------------- Helper Functions ----------------
def fetch_last_5_messages():
    with engine.connect() as conn:
        query = select(chat_history_table).order_by(desc(chat_history_table.c.timestamp)).limit(5)
        results = conn.execute(query).fetchall()
        messages = []
        for row in reversed(results):
            if row.role == "system":
                messages.append(SystemMessage(content=row.content))
            elif row.role == "user":
                messages.append(HumanMessage(content=row.content))
            elif row.role == "assistant":
                messages.append(AIMessage(content=row.content))
        return messages

def store_message(role, content):
    with engine.connect() as conn:
        conn.execute(chat_history_table.insert().values(
            role=role,
            content=content,
            timestamp=datetime.utcnow()
        ))

# ---------------- Chat Function ----------------
def get_product_info(user_input):
    # Fetch context
    context = fetch_last_5_messages()
    if not any(isinstance(m, SystemMessage) for m in context):
        context.insert(0, SystemMessage(content=SYSTEM_PROMPT))
    
    # Add current user message
    context.append(HumanMessage(content=user_input))
    
    # Run agent
    response = agent_executor.run(user_input)
    
    # Store in DB
    store_message("user", user_input)
    store_message("assistant", response)
    
    return response

In [26]:
# ---------------- Sample Usage ----------------
sample_questions = [
    "whats the category of using Medibar Personal Lubricant for Men & Women, Long Lasting Pleasure Lubricant."
]

for q in sample_questions:
    answer = get_product_info(q)
    print(f"\nYou: {q}")
    print(f"AI: {answer}")



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input:product_detailsAction: sql_db_schema
Action Input: product_details
CREATE TABLE product_details (
	product_name TEXT, 
	category TEXT, 
	price TEXT, 
	rating TEXT, 
	image_url TEXT, 
	product_url TEXT, 
	available TEXT, 
	id BIGINT GENERATED BY DEFAULT AS IDENTITY (INCREMENT BY 1 START WITH 1 MINVALUE 1 MAXVALUE 9223372036854775807 CACHE 1 NO CYCLE), 
	CONSTRAINT product_details_pkey PRIMARY KEY (id), 
	CONSTRAINT product_details_id_key UNIQUE NULLS DISTINCT (id)
)

/*
3 rows from product_details table:
product_name	category	price	rating	image_url	product_url	available	id
Care Cosmetics Care's Antidandruff shampoo with Punica Granatum Peel Extract - 100ml -Pack of 2	Personal Care	257	N/A	https://rukminim3.flixcart.com/image/546/564/xif0q/shampoo/r/e/m/200-care-s-antidandruff-shampoo-wit	https://www.shopsy.in/care-cosmetics-cares-antidandruff-shampoo-with-punica-granatum-peel-extract-10	yes	1
Care Cosme

ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: `The category of Medibar Personal Lubricant for Men & Women, Long Lasting Pleasure Lubricant is Personal Care.`
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 

In [ ]:
user_question_2 = "Who is elon musk"
answer_2 = get_product_info(user_question_2)
system = "I will give you a query and the chat response in which there will be unwanted text in the response unrelated to the query. Your task is to clean the response and provide only the relevant answer to the query without any unrelated text and without any unwanted symbols like ** or * ."
print(llm.invoke(system+answer_2).content)